In [1]:
# Intentar hacer que no haga falta que quien corra el notebook se descargue antes los datos
# Que se descarguen en la misma ubicación donde esté el notebook
# Leer solo los archivos '*.csv'
# Append de todos los ficheros
# Valorar si leer los archivos horarios o diarios (según tamaño)
# más interesante datos en tiempo real, para ver patrones de comportamiento a lo largo del día, fin de semana...
# pandas, matplotlib